In [1]:
!pip install torch sentencepiece \
  accelerate \
  bitsandbytes \
  git+https://github.com/huggingface/transformers.git@15641892985b1d77acc74c9065c332cd7c3f7d7f \
  git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/transformers.git (to revision 15641892985b1d77acc74c9065c332cd7c3f7d7f) to /tmp/pip-req-build-gvmdrslo
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-gvmdrslo
  Running command git rev-parse -q --verify 'sha^15641892985b1d77acc74c9065c332cd7c3f7d7f'
  Running command git fetch -q https://github.com/huggingface/transformers.git 15641892985b1d77acc74c9065c332cd7c3f7d7f
  Running command git checkout -q 15641892985b1d77acc74c9065c332cd7c3f7d7f
  Resolved https://github.com/huggingface/transformers.git to commit 15641892985b1d77acc74c9065c332cd7c3f7d7f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-nv33_fzv
  Running command git clone --filter=blob:none --quiet https://github.

In [2]:

!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 66.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.28.0.dev0
    Uninstalling transformers-4.28.0.dev0:
      Successfully uninstalled transformers-4.28.0.dev0


In [3]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [5]:
filename = '/content/vacancies.pt'
data = torch.load(filename)

In [6]:
from transformers import BertTokenizer as BT
from transformers import BertModel as BM
import torch

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from tqdm import tqdm

bert_tokenizer = BT.from_pretrained('bert-base-multilingual-cased')
bert_model = BM.from_pretrained('bert-base-multilingual-cased')

keywords = ['python', 'excel', 'php' 'go', 'golang', 'java', 'swift', 'kotlin', 'angular', 'javascript', 'html', 'css', 'sql', 'mongodb', 'postgresql', 'tensorflow', 'pytorch', 'keras', 'pandas', 'numpy', 'django', 'flask', 'spring', 'react', 'vue', 'angularjs', 'typescript', 'android', 'ios', 'docker', 'kubernetes', 'aws', 'azure', 'gcp', 'git', 'jenkins', 'ansible', 'terraform', 'devops', 'agile', 'scrum', 'machine learning', 'deep learning', 'data science', 'big data', 'cloud computing']

def check_keywords(description,keywords):
    for keyword in keywords:
        if keyword in description.lower():
            return True
    return False

def get_bert_embeddings(text):
    inputs = bert_tokenizer(text, return_tensors='pt', max_length=512, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    embeddings = torch.mean(outputs.last_hidden_state, dim=1)
    return embeddings


def get_similar(data,name,max_sim,words_to_find,user_embedding):
    escription_embedding = data[name]
    similarity = torch.cosine_similarity(user_embedding, escription_embedding, dim=1).item()

    flag = 1
    if(len(words_to_find) > 0):
        flag = check_keywords(data['Описание'], words_to_find)


    if(data['Сходство'] >= max_sim):
        data['Сходство'] = similarity*flag
        max_sim = data['Сходство']




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [48]:
def get_job_by_index(data, index):
    for job in data:
        if job['Index'] == index:
            return [str(job['Название вакансии']),str(job['Компания']),[str(job['Описание'])]]

In [33]:
def get_similar_jobs(user_experience):
  user_embedding = get_bert_embeddings(user_experience)


  user_promt_splited = user_experience.replace(',', '').split()
  words_to_find = []
  for x in user_promt_splited:
      if(x in keywords):
          words_to_find+=[x]


  k = 0
  for job in tqdm(data):
      job['Сходство'] = 0
      max_sim = 0
      get_similar(job,'Embeded',max_sim,words_to_find,user_embedding)


  sorted_jobs = sorted(data, key=lambda x: x['Сходство'], reverse=True)
  return sorted_jobs

In [9]:
user_experience = "Подскажи несколько вакансий для Python разработчика и требования к кандидатам".lower()

conversation = Conversation()

question = f"\nuser:{user_experience}\nbot: Вот ответ на ваш вопрос длиной не более 100 слов:"
conversation.add_user_message(question)
prompt = conversation.get_prompt(tokenizer)
output = generate(model, tokenizer, prompt, generation_config)
print(output)



Вакансии для Python инженера с опытом 1 год обычно предлагают зарплату в диапазоне от 50 до 100 тысяч рублей в месяц. Требуемые квалификации включают знание языка программирования Python, базовые понятия ООП, работу с базами данных, знание веб-технологий и фреймворков.


In [58]:
conversation = Conversation()

sorted_jobs = get_similar_jobs(user_experience)
output = ""
question = f"""promt: {get_job_by_index(data,sorted_jobs[0]['Index'])}, {get_job_by_index(data,sorted_jobs[1]['Index'])}, {get_job_by_index(data,sorted_jobs[2]['Index'])}
               \nuser:{user_experience}
               \nbot: Вот ответ на ваш вопрос длиной не более 100 слов:"""
conversation.add_user_message(question)
prompt = conversation.get_prompt(tokenizer)
output = generate(model, tokenizer, prompt, generation_config)
print("\n\nДанные из базы знаний:\n")
for vacancy in [get_job_by_index(data,sorted_jobs[0]['Index']),get_job_by_index(data,sorted_jobs[1]['Index']),get_job_by_index(data,sorted_jobs[2]['Index'])]:
  for x in vacancy:
    print(x)
  print()
print("========================LLM SAIGA===========================")
print(output)



100%|██████████| 476/476 [00:00<00:00, 29889.79it/s]


Данные из базы знаний:


QA automation
RAIDIX, Санкт-Петербург
['Условия: Полный рабочий день,опыт работы от 1 до 1 лет,высшее образование,з/п обсуждается на собеседовании руб. Требования к кандидату: Высшее/неоконченное высшее техническое образование;\r Опыт программирования на Python;\r Знакомство с Linux.\r Плюсом будет:\r Понимание теории тестирования ПО\r Опыт работы в команде с применением средств коллективной разработки (JIRA, GitLab, Docker);\r Опыт настройки CI/CD в Gitlab\r Опыт администрирования Linux.\r Знание фреймворка тестирования PyTest. Обязанности: Разработка и поддержка текущего тестового фреймворка;\r Автоматизация тестовых сценариев (PyTest, TestRail);\r Разработка новых сервисов для взаимодействия с ручными тестировщиками и разработчиками ПО;\r Встраивание автотестов в CI/CD;\r Настройка тестовых полигонов;']

Junior Software Engineer
Big Data Solutions, Санкт-Петербург
['Условия: Полный рабочий день,опыт работы от 1 до 1 лет,высшее образование,з/п по результатам 

In [61]:
user_experience = "Какие еще компетенции нужны для этой работы? Отобрази в виде списка стек для Python разработчика".lower()
question = f"""promt: {output}
               \nuser:{user_experience}
               \nbot: Вот ответ на ваш вопрос длиной не более 100 слов:"""

conversation.add_user_message(question)
prompt = conversation.get_prompt(tokenizer)
output = generate(model, tokenizer, prompt, generation_config)
print(output)

Вот список стека для Python разработчика:

1. Python: Конечно же, основной язык программирования.
2. Django: Web-фреймворк для быстрого создания веб-приложений.
3. JavaScript: Для frontend-разработки.
4. Node.js: Для backend-разработки.
5. Vue.js: Для создания интерактивных пользовательских интерфейсов.
6. MongoDB: Новая база данных.
7. HTML, CSS и адаптивная верстка: Для создания веб-страниц.
8. Docker: Для контейнерной виртуализации.
9. Git: Для управления версиями кода.
10. Flutter: Для создания мобильных приложений.

Это всего лишь некоторые из возможных технологий, которые могут быть полезны для Python разработчика.


In [64]:
user_experience = "Какие компетенции нужны для этой работы? Какие soft скилы нужно подтянуть?".lower()
question = f"""promt: {output}
               \nuser:{user_experience}
               \nbot: Вот ответ на ваш вопрос длиной не более 100 слов:"""

conversation.add_user_message(question)
prompt = conversation.get_prompt(tokenizer)
output = generate(model, tokenizer, prompt, generation_config)
print(output)

Для успешного выполнения работы в качестве Python разработчика следующие компетенции и навыки являются крайне желательными:

1. **Коммуникативные навыки**: способность чётко и ясно передавать свои идеи, мысли и планы другим людям, особенно если они работают над одним проектом.
2. **Способность к самоорганизации**: разработчик должен быть способным самостоятельно организовывать свою рабочую процедуру, планировать задачи и контролировать свой прогресс.
3. **Критический мышление**: способность критически оценивать свои идеи и решения, а также принимать исправления и улучшения.
4. **Способность к анализу и решению проблем**: разработчик должен быть способным анализировать проблемы, которые возникли в процессе работы, и найти решение для них.
5. **Способность к самообразованию**: разработчик должен быть способным самому учиться и обновлять свои навыки, особенно в свете быстрого развития технологий.
6. **Способность к работе в команде**: разработчик должен быть способным работать в команде и